In [1]:
import xarray as xr
import numpy as np
from scipy.ndimage import label, generate_binary_structure
import pandas as pd
import os
from tqdm import tqdm  # for progress bars

# Configuration
start_year = 1980
end_year = 2023
month = '01'  # January
target_latitude = -54.5  # Macquarie Island latitude
target_longitude = 159   # Macquarie Island longitude
min_ar_length = 20.0     # Minimum AR length in degrees latitude

# Path templates
input_path = "Folder/{year}/vivt{year}{month}.nc"
percentile_path = "Foldrer98%tile/98%tile/Jan.nc"
output_dir = "/Output_Directory/"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load 98th percentile threshold (same for all years)
percentile_data = xr.open_dataset(percentile_path)
vivt_abs_98_2d = percentile_data['Jan']
vivt_abs_98_2d = vivt_abs_98_2d.sel(latitude=slice(-37.5, -80))
# Prepare results container
all_results = []

# Main processing loop by year
for year in tqdm(range(start_year, end_year + 1), desc="Processing years"):
    try:
        # Load data for current year and month
        input_path = input_path_template.format(year=year, month=month)
        data = xr.open_dataset(input_path)
        data = data.sel(latitude=slice(-37.5, -80))
        
        # Get Mac Island grid coordinates
        lat = data['latitude'].values
        lon = data['longitude'].values
        ii = np.argmin(np.abs(lat - target_latitude))
        jj = np.argmin(np.abs(lon - target_longitude))
        
        # Initialize outputs for this year
        ar_mac_island_flag = np.zeros(len(data['time']))
        mac_island_vivt = np.zeros(len(data['time']))
        
        # Process each timestep
        for i in range(len(data['time'])):
            # Get current time step data
            vivt_2d = -1 * data['vivt'].values[i, :, :]
            
            # Create initial AR mask (1 where vIVT > threshold)
            ar_mask_2d = np.zeros_like(vivt_2d, dtype=int)
            ar_mask_2d[vivt_2d > vivt_abs_98_2d] = 1
            
            # Label connected AR regions
            s = generate_binary_structure(2, 2)
            ar_labels, number_of_ars = label(ar_mask_2d, structure=s)
            
            # Create filtered AR mask (only ARs >= min length)
            lon_2d, lat_2d = np.meshgrid(lon, lat)
            mac_island_affected = 0
            
            for j in range(number_of_ars):
                lat_min = np.min(lat_2d[ar_labels == j + 1])
                lat_max = np.max(lat_2d[ar_labels == j + 1])
                lat_diff = lat_max - lat_min
                
                if lat_diff >= min_ar_length:
                    # Check if this AR affects Mac Island
                    if np.any(ar_labels[ii, jj] == j + 1):
                        mac_island_affected = 1
            
            # Store results for this timestep
            ar_mac_island_flag[i] = mac_island_affected
            mac_island_vivt[i] = vivt_2d[ii, jj] if mac_island_affected else np.nan
        
        # Collect results for this year
        ar_dates = data['time'].values[ar_mac_island_flag == 1]
        ar_vivt = mac_island_vivt[ar_mac_island_flag == 1]
        
        for date, vivt in zip(ar_dates, ar_vivt):
            all_results.append({
                'Year': year,
                'Date-Time': date,
                'Absolute_vIVT': vivt
            })
            
    except Exception as e:
        print(f"Error processing year {year}: {str(e)}")
        continue

# Convert results to DataFrame and save
if all_results:
    df = pd.DataFrame(all_results)
    output_path = os.path.join(output_dir, f"AR_landfalls_Jan_{start_year}-{end_year}.xlsx")
    df.to_excel(output_path, index=False)
    print(f"\nProcessing complete. Results saved to {output_path}")
    print(f"Total AR events affecting Mac Island: {len(df)}")
else:
    print("No AR events found in the specified period.")

/home/565/ak6086/micromamba/envs/PhDAnalysis/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)

rocessing years: 100%|██████████| 44/44 [12:45<00:00, 17.41s/it]


Processing complete. Results saved to /g/data/bt44/ak6086/DDU/AR_landfalls_Jan_1980-2023.xlsx
Total AR events affecting Mac Island: 213
